In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
env = gym.make("MountainCar-v0").env

env.reset()
n_actions = env.action_space.n

#env.render("rgb_array")

In [2]:
n_actions = env.action_space.n

print(n_actions)

3


In [3]:
#create agent

from sklearn.neural_network import MLPClassifier

agent = MLPClassifier(hidden_layer_sizes=(10,10),
                      activation='tanh',
                      warm_start=True, #keep progress between .fit() calls
                      max_iter=1 #make only 1 iteration on each .fit()
                      )
#initialize agent to the dimension of state an amount of actions
agent.fit([env.reset()]*n_actions, range(n_actions))


D:\Tools\Python\Anaconda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=True)

In [4]:
def generate_session(t_max=10000):
    """
    Play game until end or for t_max ticks
    :param t_max: max ticks 
    :return:  list of states, list of actions and sum of rewards
    """
    states, actions = [], []
    total_reward = 0.

    s = env.reset()

    for t in range(t_max):
        # predict array of action probabilities

        probs = agent.predict_proba([s])[0]

        a = np.random.choice(n_actions, p=probs)
        new_s, r, done, info = env.step(a)

        states.append(s)
        actions.append(a)
        total_reward += r

        s = new_s
        if done: break

    return states, actions, total_reward

In [9]:
n_sessions = 50
percentile = 70
log = []

for i in range(50):
    # generate new sessions
    sessions = [generate_session() for __ in range(n_sessions)]

    batch_states, batch_actions, batch_rewards = map(np.array, zip(*sessions))
    
    threshold = np.percentile(batch_rewards, percentile)

    elite_states = batch_states[batch_rewards >= threshold]
    elite_actions = batch_actions[batch_rewards >= threshold]
    
    #elite_states: a list of states from top games
    #elite_actions: a list of actions from top games

    elite_states, elite_actions = map(np.concatenate, [elite_states, elite_actions])
    
    agent.fit(elite_states, elite_actions)
    print("mean reward = %.5f\tthreshold = %.1f" %(np.mean(batch_rewards), threshold))

   # show_progress(rewards_batch, log, reward_range=[0, np.max(rewards_batch)])

    if np.mean(batch_rewards) > 190:
        print("You Win! You may stop training now via KeyboardInterrupt.")

mean reward = -200.00000	threshold = -200.0


mean reward = -199.78000	threshold = -200.0


mean reward = -200.00000	threshold = -200.0


mean reward = -200.00000	threshold = -200.0


mean reward = -199.94000	threshold = -200.0


...

mean reward = -200.00000	threshold = -200.0


mean reward = -200.00000	threshold = -200.0


mean reward = -200.00000	threshold = -200.0


mean reward = -200.00000	threshold = -200.0


mean reward = -199.96000	threshold = -200.0


In [7]:
# record sessions
import gym.wrappers

env = gym.wrappers.Monitor(gym.make("MountainCar-v0"), directory="videos", force=True)

sessions = [generate_session() for _ in range(10)]
env.close();

In [8]:
# show video
from IPython.display import HTML
import os

video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1]))  # this may or may not be _last_ video. Try other indices

<video width="640" height="480" controls>
  <source src="./videos/openaigym.video.1.12848.video000008.mp4" type="video/mp4">
</video>